In [3]:
import astral
import datetime
import pytz
from pytz import timezone
import uszipcode
from uszipcode import SearchEngine


def convert_time_to_balanced_time(time=None, zipcode=10010, pretty_print=False):
    #zipcodes to lat/long to astral's location
    search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database
    zipcode_obj = search.by_zipcode(zipcode)
    #the below won't work outside the US, need to reconcile uszipcode's timezone with atral's timezone
    location = astral.Location(('name', 'region', zipcode_obj.lat, zipcode_obj.lng, "US/"+zipcode_obj.timezone, 0))
    if time is None:
        now = datetime.datetime.now()
    else:
        now = time
        
    timezone = pytz.timezone(location.timezone)
    time = timezone.localize(now)
    #print(time)
    
    beginning_of_today = datetime.datetime(time.year, time.month, time.day)
    sun = location.sun(date=beginning_of_today, local=True)
    #print(sun)
    #print()
    
    beginning_of_yesterday = datetime.datetime(time.year, time.month, time.day-1)
    sun_yesterday = location.sun(date=beginning_of_yesterday, local=True)
    
    try:
        beginning_of_tomorrow = datetime.datetime(time.year, time.month, time.day+1)
    except ValueError:
        beginning_of_tomorrow = datetime.datetime(time.year, time.month+1, 1)
    sun_tomorrow = location.sun(date=beginning_of_tomorrow, local=True)

    six_am = datetime.datetime(time.year, time.month, time.day, 6)
    six_pm = datetime.datetime(time.year, time.month, time.day, 18)

    todays_seconds = sun['sunset'] - sun['sunrise']
    #print("todays seconds: "+str(todays_seconds.total_seconds())+" or "+str(todays_seconds)+" hours.")
    
    last_nights_seconds = sun['sunrise'] - sun_yesterday['sunset']
    #print("last night's seconds: "+str(last_nights_seconds.total_seconds())+" or "+str(last_nights_seconds)+" hours.")
    
    tonights_seconds = sun_tomorrow['sunrise'] - sun['sunset']
    #print("tonight's seconds: "+str(tonights_seconds.total_seconds())+" or "+str(tonights_seconds)+" hours.")
    
    seconds_from_time_to_next_sunrise = sun['sunrise'] - time #use sun, not sun_tomorrow, because pre-dawn is same day
    #print("seconds_from_time_to_sunrise"+str(seconds_from_time_to_next_sunrise.total_seconds()))
    seconds_from_sunrise_to_time = time - sun['sunrise'] #same-day
    #print("seconds_from_sunrise_to_time"+str(seconds_from_sunrise_to_time.total_seconds()))
    seconds_from_sunset_until_time = time - sun['sunset']  #same-day, at night after sunset
    #print("seconds_from_sunset_to_time"+str(seconds_from_sunset_until_time.total_seconds()))
    
    balanced_seconds = datetime.timedelta(seconds=12*60*60)
    
    multiplier_how_much_shorter_today_is = balanced_seconds / todays_seconds
    #print("today's shortness multiplier, each second moves at x"+str(multiplier_how_much_shorter_today_is))
    
    multiplier_how_much_longer_tonight_is = (balanced_seconds / tonights_seconds)
    #print("tonight's multiplier: , each second moves at x"+str(multiplier_how_much_longer_tonight_is))
    
    multiplier_how_much_longer_last_night_was = (balanced_seconds / last_nights_seconds)
    #print("last night's multiplier: , each second moves at x"+str(multiplier_how_much_longer_last_night_was))

    if time >= sun['sunrise'] and time <= sun['sunset']:
        seconds_to_add_to_six_am = seconds_from_sunrise_to_time.total_seconds()*multiplier_how_much_shorter_today_is
        #print("seconds to add to today's 6 AM: "+str(seconds_to_add_to_six_am)+", or "+str(seconds_to_add_to_six_am/60/60)+" hours.")
        modified_time = six_am + datetime.timedelta(seconds=(seconds_to_add_to_six_am))

    elif time < sun['sunrise']: 
        seconds_to_subtract_from_this_six_am = seconds_from_time_to_next_sunrise.total_seconds()*(multiplier_how_much_longer_last_night_was)
        #print("seconds to subtract from this 6 am: "+str(seconds_to_subtract_from_this_six_am)+", or "+str(seconds_to_subtract_from_this_six_am/60/60)+" hours.")
        modified_time = six_am - datetime.timedelta(seconds=(seconds_to_subtract_from_this_six_am))
        

    elif time > sun['sunset']:        
        seconds_to_add_to_tonights_six_pm = seconds_from_sunset_until_time.total_seconds()*multiplier_how_much_longer_tonight_is
        #print("seconds to add to tonight's 6 PM: "+str(seconds_to_add_to_tonights_six_pm)+", or "+str(seconds_to_add_to_tonights_six_pm/60/60)+" hours.")
        modified_time = six_pm + datetime.timedelta(seconds=(seconds_to_add_to_tonights_six_pm))
    if pretty_print == False:
        return modified_time 
    else:
        return modified_time.strftime("%I:%M:%S %p")
    
    
def clock_conversion(zipcode=10010):
    search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database
    zipcode_obj = search.by_zipcode(zipcode)
    today = datetime.datetime.utcnow()
    print("balanced calendar for "+zipcode_obj.post_office_city+", "+today.strftime("%Y/%m/%d"))
    print("  old   :  balanced")
    hour = 0
    
    while hour <= 23:
        time_to_show = convert_time_to_balanced_time(time=datetime.datetime(today.year, today.month, today.day,hour,0,0), pretty_print=True, zipcode=zipcode)
        if hour == 0:
            print("12:00 AM = "+time_to_show)
        elif hour < 10:
            print(str(hour)+":00 AM  = "+time_to_show)
        elif hour < 12:
            print(str(hour)+":00 AM = "+time_to_show)
        elif hour == 12:
            print("12:00 PM = "+time_to_show)
        elif hour < 22:
            print(str(hour-12)+":00 PM  = "+time_to_show)
        else:
            print(str(hour-12)+":00 PM = "+time_to_show)
        hour +=1    

In [6]:
clock_conversion(zipcode=4005)

balanced calendar for Austin, TX, 2019/08/31
  old   :  balanced
12:00 AM = 10:21:01 PM
1:00 AM  = 11:25:34 PM
2:00 AM  = 12:30:08 AM
3:00 AM  = 01:34:41 AM
4:00 AM  = 02:39:14 AM
5:00 AM  = 03:43:47 AM
6:00 AM  = 04:48:20 AM
7:00 AM  = 05:52:53 AM
8:00 AM  = 06:49:57 AM
9:00 AM  = 07:46:05 AM
10:00 AM = 08:42:13 AM
11:00 AM = 09:38:21 AM
12:00 PM = 10:34:29 AM
1:00 PM  = 11:30:36 AM
2:00 PM  = 12:26:44 PM
3:00 PM  = 01:22:52 PM
4:00 PM  = 02:19:00 PM
5:00 PM  = 03:15:08 PM
6:00 PM  = 04:11:16 PM
7:00 PM  = 05:07:24 PM
8:00 PM  = 06:04:03 PM
9:00 PM  = 07:08:26 PM
10:00 PM = 08:12:49 PM
11:00 PM = 09:17:13 PM


15th
```'noon': datetime.datetime(2018, 11, 15, 11, 52, 50, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>), 'sunset': datetime.datetime(2018, 11, 15, 16, 57, 29, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>), 'sunrise': datetime.datetime(2018, 11, 15, 6, 48, 10, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)}```

16th
```'noon': datetime.datetime(2018, 11, 16, 11, 53, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>), 'sunset': datetime.datetime(2018, 11, 16, 16, 56, 47, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>), 'sunrise': datetime.datetime(2018, 11, 16, 6, 49, 14, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)}```


In [213]:
from datetime import timedelta


In [7]:
#???

#6:00 AM real time
print(convert_time_to_balanced_time(time=datetime.datetime(2019,12,22,6,0,0), pretty_print=True, zipcode=4005))

05:02:44 AM


In [231]:
#YES

#ten seconds before real sunrise, should be subtracting around 9 seconds
convert_time_to_balanced_time(time=datetime.datetime(2018,11,16,6,48,46), pretty_print=True, zipcode=94546)

'05:59:51AM'

In [232]:
#YES

#sunrise
convert_time_to_balanced_time(time=datetime.datetime(2018,11,16,6,48,56), pretty_print=True, zipcode=94546)

'06:00:00AM'

In [233]:
#YES

#'solar noon' should output 12:00
#time that 11/15 11:52:57 should read: 12 PM
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,11,52,47), zipcode=94546,pretty_print=True)

'12:00:00PM'

In [234]:
#YES

#12:00PM, not noon
#time that 12:00 should read is 12:08:31
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,12,0,0), zipcode=94546,pretty_print=True)

'12:08:31PM'

In [235]:
#just before sunset
#sunrise to sunset is 6:48:56 on the 15th to 16:57:47 on the 15th
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,16,57,41), zipcode=94546,pretty_print=True)

'05:59:58PM'

In [236]:
#CORRECT

#sunset
#sunrise to sunset is 6:48:56 on the 15th to 16:57:47 on the 15th
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,16,57,42), zipcode=94546,pretty_print=True)

'06:00:00PM'

In [237]:
#10 seconds after sunset - should be 8.2 seconds added or so
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,16,57,52), zipcode=94546,pretty_print=True)

'06:00:08PM'

In [242]:
#9PM
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,21,0,0), zipcode=94546,pretty_print=True)
#USE TIMEDELTA TO DETERMINE NUMBER OF SECONDS, ETC

'09:29:52PM'

In [238]:
#'midnight'
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,23,53,19), zipcode=94546,pretty_print=True)
#USE TIMEDELTA TO DETERMINE NUMBER OF SECONDS, ETC

'12:00:00AM'

In [239]:
#just before real midnight
convert_time_to_balanced_time(time=datetime.datetime(2018,11,15,23,59,59), zipcode=94546,pretty_print=True)
#need to calculate by hand what time should be

'12:05:46AM'

In [240]:
#just at real midnight
#time should read XXX
convert_time_to_balanced_time(time=datetime.datetime(2018,11,16,0,0,0), pretty_print=True, zipcode=94546)

'12:05:47AM'

In [241]:
#just after real midnight
#time should read XXX
convert_time_to_balanced_time(time=datetime.datetime(2018,11,16,0,0,10), pretty_print=True, zipcode=94546)

'12:05:56AM'

In [225]:
#math dump
timedelta(hours=6,minutes=48, seconds=56) + timedelta(hours=0,minutes=0, seconds=0)

datetime.timedelta(0, 24536)

In [201]:
7.38333333333333333*60*60

26580.0

In [20]:
from datetime import timedelta
timedelta(hours=6,minutes=48, seconds=56) - timedelta(hours=1,minutes=15, seconds=0)

datetime.timedelta(0, 20036)